JW Test Code

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("../")
sys.path.append("../..")

# Data Integration

## Data Preparation

- 테스트를 위해 data_partial_numeric데이터와 data_partial_mix 데이터를 생성함
- Meta Data 를 생성하고 이 데이터의 각 Column 정보에 의거하여 Data 를 integration 함
- 향후 Integration 하는 모듈을 고도화 하여 사용자의 파라미터에 의해 각 컬럼, 혹은 테이블 단위로 데이터들이 개선되어 Integration되도록 함
- 현재 파이프라인은 
integrated_data = data_int.simple_integration(column_meta['overlap_duration'])
integrated_data_resample = data_int.restructured_data_with_new_frequency(re_frequency, column_characteristics)
integrated_data_resample_fillna = data_int.restructured_data_fillna(integrated_data_resample, column_characteristics,re_frequency )
1. 그냥 결합한 후
2. 새로운 프리컨시에 맞게 각 컬럼들이 각각의 method로  up, down sampling취한 후
3. 각 컬럼당 Nan값이 과도하지 않은 경우 다시 NA를 없앰

요러한 파이프라인

### Make 3 Numeric Test Data with different frequencies

In [2]:
r_0 = pd.date_range(start='1/1/2018', end= '1/02/2018', freq='10T')
r_1 = pd.date_range(start='1/1/2018', end= '1/02/2018', freq='7T')
r_2 =pd.date_range(start='1/1/2018', end= '1/02/2018', freq='3T')
data_0 = {'datetime': r_0,
          'data0':np.random.randint(0, 100, size=(len(r_0)))}
data_1 = {'datetime': r_1,
          'data1':np.random.randint(0, 100, size=(len(r_1)))}
data_2 = {'datetime': r_2,
          'data2':np.random.randint(0, 100, size=(len(r_2)))}

df0 = pd.DataFrame (data = data_0).set_index('datetime')
df1 = pd.DataFrame (data = data_1).set_index('datetime')
df2 = pd.DataFrame (data = data_2).set_index('datetime')
data_partial_numeric = {0:df0, 1:df1, 2:df2}
data_partial_numeric

{0:                      data0
 datetime                  
 2018-01-01 00:00:00     23
 2018-01-01 00:10:00     34
 2018-01-01 00:20:00     82
 2018-01-01 00:30:00     43
 2018-01-01 00:40:00      1
 ...                    ...
 2018-01-01 23:20:00     39
 2018-01-01 23:30:00     38
 2018-01-01 23:40:00     67
 2018-01-01 23:50:00     13
 2018-01-02 00:00:00     49
 
 [145 rows x 1 columns],
 1:                      data1
 datetime                  
 2018-01-01 00:00:00     57
 2018-01-01 00:07:00     60
 2018-01-01 00:14:00     99
 2018-01-01 00:21:00     86
 2018-01-01 00:28:00     88
 ...                    ...
 2018-01-01 23:27:00     98
 2018-01-01 23:34:00     58
 2018-01-01 23:41:00     55
 2018-01-01 23:48:00     80
 2018-01-01 23:55:00     22
 
 [206 rows x 1 columns],
 2:                      data2
 datetime                  
 2018-01-01 00:00:00     92
 2018-01-01 00:03:00     15
 2018-01-01 00:06:00     53
 2018-01-01 00:09:00     10
 2018-01-01 00:12:00     69
 ...         

### Make +1 Category Test Data with different frequencies

In [3]:
import random
original_list=['apple','orange','pineapple']
r_3 =pd.date_range(start='1/1/2018', end= '1/02/2018', freq='8T')
sample_list = random.choices (original_list, k=len(r_3))
data_3 = {'datetime':r_3, 'data3':sample_list}
df3 = pd.DataFrame (data = data_3).set_index('datetime')
data_partial_mix = {0:df0, 1:df1, 2:df2, 3:df3}

## Integration Example

### 1. All Numeric Data

#### 1-1. Get Integration meta

In [4]:
#Make Metadata
from KETIPreDataIntegration.meta import partialDataInfo
partial_data_info = partialDataInfo.PartialData(data_partial_numeric)
print(partial_data_info.column_meta)
print(partial_data_info.partial_frequency_info)
print(partial_data_info.integrated_data_type)

IntType
IntType
IntType
[600, 420, 180]
{'overlap_duration': {'start_time': Timestamp('2018-01-01 00:00:00'), 'end_time': Timestamp('2018-01-01 23:55:00')}, 'column_characteristics': {'data0': {'column_name': 'data0', 'column_frequency': Timedelta('0 days 00:10:00'), 'column_type': dtype('int64'), 'occurence_time': 'Continuous', 'pointDependency': 'Yes', 'upsampling_method': 'mean', 'downsampling_method': 'mean'}, 'data1': {'column_name': 'data1', 'column_frequency': Timedelta('0 days 00:07:00'), 'column_type': dtype('int64'), 'occurence_time': 'Continuous', 'pointDependency': 'Yes', 'upsampling_method': 'mean', 'downsampling_method': 'mean'}, 'data2': {'column_name': 'data2', 'column_frequency': Timedelta('0 days 00:03:00'), 'column_type': dtype('int64'), 'occurence_time': 'Continuous', 'pointDependency': 'Yes', 'upsampling_method': 'mean', 'downsampling_method': 'mean'}}}
{'frequency_list': [600, 420, 180], 'min_frequency': 180, 'max_frequency': 600, 'frequency_is_same': False, 'aver

#### 1-2. Simple Integration

In [5]:
# Integration
from KETIPreDataIntegration.meta import data_integration
data_int = data_integration.DataIntegration(data_partial_numeric)
print(partial_data_info.column_meta['overlap_duration'])
simpleIntegratedData = data_int.simple_integration(partial_data_info.column_meta['overlap_duration'])
simpleIntegratedData

{'start_time': Timestamp('2018-01-01 00:00:00'), 'end_time': Timestamp('2018-01-01 23:55:00')}


,data0,data1,data2
datetime,,,
2018-01-01 00:00:00,23.0,57.0,92.0
2018-01-01 00:03:00,NaN,NaN,15.0
2018-01-01 00:06:00,NaN,NaN,53.0
2018-01-01 00:07:00,NaN,60.0,NaN
2018-01-01 00:09:00,NaN,NaN,10.0
...,...,...,...
2018-01-01 23:48:00,NaN,80.0,24.0
2018-01-01 23:50:00,13.0,NaN,NaN
2018-01-01 23:51:00,NaN,NaN,24.0


#### 1-3. data reconstruction with new frequency

In [6]:
import datetime

partial_data_type = partial_data_info.integrated_data_type
column_characteristics = partial_data_info.column_meta['column_characteristics']
#column_characteristics['data0']['upsampling_method']=np.interp
import datetime
re_frequency_min = 3
re_frequency_sec = re_frequency_min*60
re_frequency = datetime.timedelta(seconds= re_frequency_sec)

integrated_data_resample = data_int.restructured_data_with_new_frequency(re_frequency, column_characteristics)
integrated_data_resample

,data0,data1,data2
datetime,,,
2018-01-01 00:00:00,23.0,57.0,92.0
2018-01-01 00:03:00,NaN,NaN,15.0
2018-01-01 00:06:00,NaN,60.0,53.0
2018-01-01 00:09:00,34.0,NaN,10.0
2018-01-01 00:12:00,NaN,99.0,69.0
...,...,...,...
2018-01-01 23:42:00,NaN,NaN,30.0
2018-01-01 23:45:00,NaN,NaN,87.0
2018-01-01 23:48:00,13.0,80.0,24.0


In [7]:
partial_data_info.column_meta['column_characteristics']

{'data0': {'column_name': 'data0',
  'column_frequency': Timedelta('0 days 00:10:00'),
  'column_type': dtype('int64'),
  'occurence_time': 'Continuous',
  'pointDependency': 'Yes',
  'upsampling_method': 'mean',
  'downsampling_method': 'mean'},
 'data1': {'column_name': 'data1',
  'column_frequency': Timedelta('0 days 00:07:00'),
  'column_type': dtype('int64'),
  'occurence_time': 'Continuous',
  'pointDependency': 'Yes',
  'upsampling_method': 'mean',
  'downsampling_method': 'mean'},
 'data2': {'column_name': 'data2',
  'column_frequency': Timedelta('0 days 00:03:00'),
  'column_type': dtype('int64'),
  'occurence_time': 'Continuous',
  'pointDependency': 'Yes',
  'upsampling_method': 'mean',
  'downsampling_method': 'mean'}}

#### 1-4 FillNA based on upsampling condition

In [8]:
 integrated_data_resample_fillna = data_int.restructured_data_fillna(integrated_data_resample, column_characteristics,re_frequency )

In [9]:
 integrated_data_resample_fillna

,data0,data1,data2
datetime,,,
2018-01-01 00:00:00,23.000000,57.000000,92.0
2018-01-01 00:03:00,26.666667,58.500000,15.0
2018-01-01 00:06:00,30.333333,60.000000,53.0
2018-01-01 00:09:00,34.000000,79.500000,10.0
2018-01-01 00:12:00,50.000000,99.000000,69.0
...,...,...,...
2018-01-01 23:42:00,49.000000,63.333333,30.0
2018-01-01 23:45:00,31.000000,71.666667,87.0
2018-01-01 23:48:00,13.000000,80.000000,24.0


2. ### Mixing Data

In [ ]:
partial_data_info = partialDataInfo.PartialData(data_partial_mix)


print(partial_data_info.integrated_data_type)
print(partial_data_info.partial_frequency_info)
print(partial_data_info.column_meta)

In [ ]:
data_set = data_partial_mix
from KETIPreDataIntegration.meta import partialDataInfo
partial_data_info = partialDataInfo.PartialData(data_set)

#### 2-0. data integration by one method

In [ ]:
# Integration
from KETIPreDataIntegration.meta import data_integration
data_set = data_partial_numeric
partial_data_info = partialDataInfo.PartialData(data_set)
data_it = data_integration.DataIntegration(data_set)
integrated_data_resample = data_it.dataIntegrationByMeta(re_frequency, partial_data_info.column_meta)

In [ ]:
integrated_data_resample

### Easy integration Method

In [ ]:
from KETIPreDataIntegration.clustDataIntegration import ClustIntegration

data = ClustIntegration().getIntegratedDataSetByMeta(data_set, re_frequency_min)